In [ ]:
# Exemple du prof, légèrement modifié pour faire de 
# la sélection de features à partir du test du chi 2

# Ici on utilise la libraire Mllib plut que spark ml.
# Tout le travail se fait donc sur les RDD.

# NEXT : d'autres algo de sélections de features,
# d'autres algos de classifiaction et combinaison avec 
# les prétraitements des autres fichiers

In [1]:
import numpy as np
from random import randint
from  pyspark.mllib.classification import NaiveBayes
from functools import partial
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint

# Rajouté afin de ne pas avoir à recharger les modules comme
# loadFiles lorsqu'ils sont modifiés. Les modifications sont ici
# automatiquement prises en compte.
%load_ext autoreload
%autoreload 2


In [2]:
def createBinaryLabeledPoint(doc_class,dictionary):
    # Version ultra-basique sans pré-traitement
	words=doc_class[0].strip().split(' ')
    
	#create a binary vector for the document with all the words that appear (0:does not appear,1:appears)
	#we can set in a dictionary only the indexes of the words that appear
	#and we can use that to build a SparseVector
	vector_dict={}
	for w in words:
		vector_dict[dictionary[w]]=1
        
	return LabeledPoint(doc_class[1], SparseVector(len(dictionary),vector_dict))


In [3]:
def Predict(name_text,dictionary,model):
	words=name_text[1].strip().split(' ')
	vector_dict={}
	for w in words:
		if(w in dictionary):
			vector_dict[dictionary[w]]=1
	return (name_text[0], model.predict(SparseVector(len(dictionary),vector_dict)))

In [4]:
# loadFilesPartial est identique à loadFiles mais
# permet de charger uniquement les n premiers fichiers positifs
# et les n premiers négatifs.

# NEXT : faire en sorte qu'elle choisisse aléatoirement les 
# fichiers et non les premiers

import loadFilesPartial as lfp
data,Y=lfp.loadLabeled("./data/train", 100)
print len(data)

200


In [5]:
dataRDD=sc.parallelize(data,numSlices=16)
lists=dataRDD.map(lambda x:list(set(x.strip().split(' ')))).collect()
all=[]
for l in lists:
    all.extend(l)
dict=set(all)

In [6]:
#it is faster to know the position of the word if we put it as values in a dictionary
dictionary={}
for i,word in enumerate(dict):
	dictionary[word]=i
#we need the dictionary to be available AS A WHOLE throughout the cluster
dict_broad=sc.broadcast(dictionary)

In [7]:
len(dict_broad.value)

10775

In [8]:
#build labelled Points from data
data_class=zip(data,Y)#if a=[1,2,3] & b=['a','b','c'] then zip(a,b)=[(1,'a'),(2, 'b'), (3, 'c')]
dcRDD=sc.parallelize(data_class,numSlices=16)
#get the labelled points
labeledRDD=dcRDD.map(partial(createBinaryLabeledPoint,dictionary=dict_broad.value))

In [21]:
# Définition du feature selector : on retient les 6000 features
# les plus pertinentes au sens du Chi 2 sur la base d'entraînement

# STOP : CA NE MARCHE PAS 
# Je ne comprends pas pourquoi mais j'obtiens une runTime error lorsque j'essaie d'entrâiner
# le modèle sur les features sélectionner. Il semblerait qu'il soit impossible
# d'appeler un transformer dans le map (deux cases en dessous)

# La solution à implémenter est de calculer la pertinence de chaque feature (module 
# statistics de mllib) et ainsi obtenir les positions des plus pertinentes pour ensuite ne
# garder que celles-ci dans la rdd. On pert l'automatisme du selector.

from pyspark.mllib.feature import ChiSqSelector
selector = ChiSqSelector(6000)
transformer = selector.fit(labeledRDD)
s_broad = sc.broadcast(transformer)

Exception: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transforamtion. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.

In [14]:
def createBLPselect(doc_class,dictionary,transformer):
    # Version ultra-basique sans pré-traitement
	words=doc_class[0].strip().split(' ')
    
	#create a binary vector for the document with all the words that appear (0:does not appear,1:appears)
	#we can set in a dictionary only the indexes of the words that appear
	#and we can use that to build a SparseVector
	vector_dict={}
	for w in words:
		vector_dict[dictionary[w]]=1
        
	return LabeledPoint(doc_class[1], transformer.transform(SparseVector(len(dictionary),vector_dict)))


In [19]:
selectedRDD=dcRDD.map(partial(createBLPselect,dictionary=dict_broad.value,transformer=s_broad.value))

In [20]:
selectedRDD.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 48.0 failed 1 times, most recent failure: Lost task 0.0 in stage 48.0 (TID 723, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/1.5.1/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/Cellar/apache-spark/1.5.1/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/Cellar/apache-spark/1.5.1/libexec/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/Cellar/apache-spark/1.5.1/libexec/python/pyspark/rdd.py", line 1295, in takeUpToNumLeft
    yield next(iterator)
  File "<ipython-input-14-30afd

In [ ]:
#Train NaiveBayes
model=NaiveBayes.train(labeledRDD)
#broadcast the model
mb=sc.broadcast(model)

In [ ]:
test,names=lfp.loadUknown('./data/test',100)

In [ ]:
name_text=zip(names,test)
#for each doc :(name,text):
#apply the model on the vector representation of the text
#return the name and the class
predictions=sc.parallelize(name_text).map(partial(Predict,dictionary=dict_broad.value,model=mb.value)).collect()


In [ ]:
output=file('./classificationsAllFeat.txt','w')
for x in predictions:
	output.write('%s\t%d\n'%x)
output.close()
